### 과제
* medium, source에 대한 예산 집행하려 했을 때 임의의 금액을 배정 -> 1억 ~ 5억 랜덤 배정
* 예산 컬럼 기준으로 효율성 준석
    * 예산, pageviews, hits, visits에 따른 효율성 지표 생성
* plotly 사용하여 어떤 트래픽이 가장 효율적이고 비효율적인지 시각화

In [164]:
import pandas as pd
import numpy as np
import plotly.express as px

In [165]:
df = pd.read_csv('/Users/sejinmoon/Desktop/Coding/BDA/10기_지표설계/ga.csv')
df.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Direct,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6.194190e+18,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",6194193421514403509_1472843572,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""11...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1472843572,1,1472843572
1,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",5.327170e+18,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",5327166854580374902_1472844906,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""10...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472844906,3,1472844906
2,Referral,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8.885050e+18,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",8885051388942907862_1472827393,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""13"", ""pageviews"": ""11...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1472827393,7,1472827393
3,Referral,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1.854680e+17,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",0185467632009737931_1472846398,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""13"", ""pageviews"": ""12...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1472846398,6,1472846398
4,Referral,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3.244890e+18,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",3244885836845029978_1472824817,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""17"", ""pageviews"": ""14...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1472824817,4,1472824817


In [167]:
import json

#totals 풀기
df['totals'] = df['totals'].apply(json.loads)

#trafficSource -> source, medium dict 객체로 변환해서 가지고 와야 한다.
df['trafficSource'] = df['trafficSource'].apply(json.loads)

#수치형 컬럼을 추출을 해야 한다.
df['pageviews'] = df['totals'].apply(lambda x : int(x.get('pageviews'), 0))
df['hits'] = df['totals'].apply(lambda x : int(x.get('hits'), 0))
df['visits'] = df['totals'].apply(lambda x : int(x.get('visits'), 0))

#campaign, source, medium을 가지고 오자.
df['campaign']=df['trafficSource'].apply(lambda x : x.get('campaign','(not set)'))
df['source']=df['trafficSource'].apply(lambda x : x.get('source','(not set)'))
df['medium']=df['trafficSource'].apply(lambda x : x.get('medium','(not set)'))

#집계
agg_df_sm = df.groupby(['date', 'source', 'medium'])[['pageviews', 'hits', 'visits']].sum().reset_index()

In [168]:
agg_df_sm

,date,source,medium,pageviews,hits,visits
0,20160801,(direct),(none),341,441,8
1,20160801,basecamp.com,referral,49,69,1
2,20160801,gatewaycdi.com,referral,14,18,1
3,20160801,google,organic,216,372,7
4,20160801,mall.googleplex.com,referral,428,568,16
...,...,...,...,...,...,...
1753,20170801,google,cpc,20,28,1
1754,20170801,google,cpm,5,5,1
1755,20170801,google,organic,244,319,6
1756,20170801,mail.google.com,referral,16,16,1


In [ ]:
# 고유 조합 추출
group_keys = agg_df_sm[['medium', 'source']].drop_duplicates()
np.random.seed(0)

# 예산 배정: 각 조합에 랜덤으로 1억~5억 사이 금액을 배정
group_keys['budget'] = np.random.randint(10000000, 500000000, size=len(group_keys))


In [ ]:
# 예산 배정된 조합을 원래 데이터프레임에 병합
agg_df_sm = agg_df_sm.merge(group_keys, on=['medium', 'source'], how='left')

In [ ]:
# 각 지표별 예산 대비 비율 계산
agg_df_sm['pageviews_per_budget'] = agg_df_sm['pageviews'] / agg_df_sm['budget']
agg_df_sm['hits_per_budget'] = agg_df_sm['hits'] / agg_df_sm['budget']
agg_df_sm['visits_per_budget'] = agg_df_sm['visits'] / agg_df_sm['budget']

In [ ]:
# 각 지표별 가중치 부여
# 가중치 부여 후 효율성 점수 계산
agg_df_sm['weighted_efficiency_score'] = (
    0.4 * agg_df_sm['pageviews_per_budget'] +
    0.2 * agg_df_sm['hits_per_budget'] +
    0.6 * agg_df_sm['visits_per_budget']
)

In [173]:
agg_df_sm

,date,source,medium,pageviews,hits,visits,budget,pageviews_per_budget,hits_per_budget,visits_per_budget,weighted_efficiency_score
0,20160801,(direct),(none),341,441,8,219652396,1.552453e-06,2.007718e-06,3.642118e-08,1.044377e-06
1,20160801,basecamp.com,referral,49,69,1,408764591,1.198734e-07,1.688013e-07,2.446396e-09,8.317746e-08
2,20160801,gatewaycdi.com,referral,14,18,1,397360373,3.523250e-08,4.529893e-08,2.516607e-09,2.466275e-08
3,20160801,google,organic,216,372,7,414868288,5.206472e-07,8.966701e-07,1.687282e-08,3.977166e-07
4,20160801,mall.googleplex.com,referral,428,568,16,451365315,9.482341e-07,1.258404e-06,3.544801e-08,6.522433e-07
...,...,...,...,...,...,...,...,...,...,...,...
1753,20170801,google,cpc,20,28,1,427693031,4.676251e-08,6.546752e-08,2.338126e-09,3.320138e-08
1754,20170801,google,cpm,5,5,1,228660017,2.186653e-08,2.186653e-08,4.373305e-09,1.574390e-08
1755,20170801,google,organic,244,319,6,414868288,5.881385e-07,7.689187e-07,1.446242e-08,3.977166e-07
1756,20170801,mail.google.com,referral,16,16,1,99739541,1.604178e-07,1.604178e-07,1.002611e-08,1.022664e-07


In [ ]:
# 효율성이 가장 높은 5개 트래픽
top_traffic = agg_df_sm[['source', 'medium', 'weighted_efficiency_score']].sort_values(by='weighted_efficiency_score', ascending=False).head(5)

# 막대 그래프
fig = px.bar(top_traffic, 
             x='weighted_efficiency_score', 
             y='source', 
             color='medium', 
             orientation='h', 
             title='상위 트래픽',
             labels={'weighted_efficiency_score': 'Efficiency Score', 'source': 'Source & Medium'})
fig.show()


In [ ]:
# 효율성이 가장 낮은 5개 트래픽
bottom_traffic = agg_df_sm[['source', 'medium', 'weighted_efficiency_score']].sort_values(by='weighted_efficiency_score').head(5)

# 막대 그래프
fig = px.bar(bottom_traffic, 
             x='weighted_efficiency_score', 
             y='source', 
             color='medium', 
             orientation='h', 
             title='하위 트래픽',
             labels={'weighted_efficiency_score': 'Efficiency Score', 'source': 'Source & Medium'})
fig.show()

## 결론
* 효율성이 가장 좋은 조합
    * Source : facebook.com
    * medium : referral
* 효율성이 가장 나쁜 조합
    * Source : groups.google.com
    * medium : referral